<a href="https://colab.research.google.com/github/Nakul24-1/Viz-for-ML/blob/main/D3_in_Jupyter/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 1: CS-GY 9223
## Exploring 20 NewsGroups

In this homework, you will write a D3 Visualization in Javascript and integrate it in Jupyter Notebook.

The goal of this exercise is to explore the **20 News Groups dataset**, a popular machine learning dataset that contains news articles grouped in 20 topics. Your visualization should receive the dataset and display a bar chart with the top most frequent words in the dataset. The user should be able to filter the data based on topic (for example, by clicking in checkboxes, selecting from a drop down menu, etc.). The user should also be able to export the selected documents from the selected topic back to Python (using a button).

In summary, your visualization should have the following capabilities:
- Display a bar chart with the top K words in the document collection
- Enable the user to filter the documents based on topic, and display a bar chart with the frequency of the top K words from that topic.
- Export the documents from the selected news topic back to python (as a list of strings).
- The visualization has to be integrated with python. The API should have two functions:
  - `plot_top_words(documents, K) # plot top K words using D3 and Javascript`
  - `get_exported_documents() # get the exported documents back to python`
  
Example of the resulting visualization:
<img src="HW_Vis.png" width = "500px" height="100px"/>

### Accessing the data

The data should be accessed from sklearn. In this section we show an example of code for accessing the documents and the document classes.

In [ ]:
# Fetching the data
from sklearn.datasets import fetch_20newsgroups
import numpy as np
newsgroups = fetch_20newsgroups(subset='test')

# getting the topic ids
topic_idx = np.array(newsgroups.target, dtype=int)

# getting the unique topic names
topic_names = np.array(newsgroups.target_names)

# getting the list of documents
documents = list(newsgroups.data)

# getting the list of topics (in the same order as documents)
topics = list(topic_names[topic_idx])

These are the 20 topics in the dataset:

In [ ]:
print(topic_names)

['alt.atheism' 'comp.graphics' 'comp.os.ms-windows.misc'
 'comp.sys.ibm.pc.hardware' 'comp.sys.mac.hardware' 'comp.windows.x'
 'misc.forsale' 'rec.autos' 'rec.motorcycles' 'rec.sport.baseball'
 'rec.sport.hockey' 'sci.crypt' 'sci.electronics' 'sci.med' 'sci.space'
 'soc.religion.christian' 'talk.politics.guns' 'talk.politics.mideast'
 'talk.politics.misc' 'talk.religion.misc']


The documents and document topics are assigned to the variables *documents* and *topics*. We print some document examples below.

In [ ]:
for i in range(1, 3):
    print("Topic: {}".format(topics[i]))
    print("-"*60)
    print("Document:")
    print(documents[i])
    print("="*60)
    print("")

Topic: comp.windows.x
------------------------------------------------------------
Document:
From: Rick Miller <rick@ee.uwm.edu>
Subject: X-Face?
Organization: Just me.
Lines: 17
Distribution: world
NNTP-Posting-Host: 129.89.2.33
Summary: Go ahead... swamp me.  <EEP!>

I'm not familiar at all with the format of these "X-Face:" thingies, but
after seeing them in some folks' headers, I've *got* to *see* them (and
maybe make one of my own)!

I've got "dpg-view" on my Linux box (which displays "uncompressed X-Faces")
and I've managed to compile [un]compface too... but now that I'm *looking*
for them, I can't seem to find any X-Face:'s in anyones news headers!  :-(

Could you, would you, please send me your "X-Face:" header?

I *know* I'll probably get a little swamped, but I can handle it.

	...I hope.

Rick Miller  <rick@ee.uwm.edu> | <ricxjo@discus.mil.wi.us>   Ricxjo Muelisto
Send a postcard, get one back! | Enposxtigu bildkarton kaj vi ricevos alion!
          RICK MILLER // 16203 WOOD

In [ ]:
documents.insert(0, ' '.join(documents))
topics.insert(0, "all")
topic_names = np.insert(topic_names, 0, "all")

In [ ]:
from collections import Counter
import glob
import random
import json
from IPython.display import display, HTML
import json

In [ ]:
ls = []
count = []
word = []
dicts = []
for x in range (len(documents)):
  if topics[x] in topic_names:
    for i in documents[x].split():
      ls.append(i)
dic = Counter(ls)
n = 10
for i in range(n):
  dicts.append((dic.most_common(n)[i][1],(dic.most_common(n)[i][0])))
  count.append(dic.most_common(n)[i][1])
  word.append((dic.most_common(n)[i][0]))
docs = {
    "array": count,
    "word": word
}

In [ ]:
def target_func(comm, msg):
  val1 = msg['content']['data'][0]
  print(val1)
  val2 = msg['content']['data'][1]
  topic= [val1]
  cntr = 0;
  ls = []
  count = []
  word = []
  dicts = []
  
  for x in range (len(documents)):
    if topics[x] in topic:
      if cntr < 6:
        print(documents[x])
      for i in documents[x].split():
        ls.append(i)
  dic = Counter(ls)
  n = int(val2)
  print(n)
  for i in range(n):
    dicts.append((dic.most_common(n)[i][1],(dic.most_common(n)[i][0])))
    count.append(dic.most_common(n)[i][1])
    word.append((dic.most_common(n)[i][0]))

  docs = {
      "array": count,
      "word": word
}
  comm.send(docs,None)
get_ipython().kernel.comm_manager.register_target('my_comm_target', target_func)

In [ ]:
from IPython.display import HTML
from IPython.display import Javascript

display(HTML("""
<html>
  <head>
    <script src="https://d3js.org/d3.v6.min.js"></script>
</head>
  <body>
    
    <div id = "list">
    <label>Number of words: </label>
    <input type="range" min="1" max="20" step="1" value="10" id="val" oninput="this.nextElementSibling.value = this.value">
        <output>10</output>
    <br>
    Select document category-
    </p>
    </div>
    <div id="disp">
    <p>
    
    </p>
    </div>
    <div id = "exp">
      <button type="button" id = "export">Export</button>
    </div>
  <script>
{{
function getArray(object) {{
    return Object.keys(object).reduce(function (r, k) {{
        object[k].forEach(function (a, i) {{
            r[i] = r[i] || {{}};
            r[i][k] = a;
        }});
        return r;
    }}, []);
}}
function getObject(array) {{
    return array.reduce(function (r, o, i) {{
        Object.keys(o).forEach(function (k) {{
            r[k] = r[k] || [];
            r[k][i] = o[k];
        }});
        return r;
    }}, {{}});
}}
function bar_chart(data)
{{
  const data2 = getArray(data)
  var result = Object.entries(data);
  var word = data.word
  const margin = ({{top: 20, bottom: 45, left: 75, right: 30}})  
  const width = 1000
  const height = 500
  const acWidth = width - margin.left - margin.right
  const acHeight = height - margin.top - margin.bottom
  var x = d3.scaleBand()
    .domain(word)
    .range([0, acWidth])
    .padding(0.2)
  var y = d3.scaleLinear()
    .domain([0, d3.max(data.array)])
    .range([acHeight,0])
  const xAxis = d3.axisBottom(x);
  const yAxis = d3.axisLeft(y);
  const svg = d3.select("#disp").append("svg")
    .attr('width', width)
    .attr('height', height);
  const g = svg.append("g")
      .attr('transform', `translate(${{margin.left}}, ${{margin.top}})`);
  g.selectAll('rect')
    .data(data2)
    .join('rect')
      .attr('x', d => x(d.word))
      .attr('y', d => y(d.array)) 
      .attr('height', d => acHeight - y(d.array))
      .attr('width', x.bandwidth())
      .attr('fill', 'blue');
  g.append('g')
      .call(yAxis)
  g.append('g')
      .attr('transform', `translate(0, ${{acHeight}})`)
      .call(xAxis)
      .append('text')
      .attr('fill', 'black')
      .attr('font-family', 'sans-serif')
      .attr('y', acHeight / 2)
      .attr('x', 40)
      .text("Count");
  }}
  bar_chart({data})
    var allGroup = ['all', 'alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
    var wordc = 10
    var dropdownButton = d3.select("#list")
      .append('select').attr("value", function (d) {{ return d; }})
    var num_words = d3.select("#val").on("input", function() {{
      wordc = this.value;
      }});
    dropdownButton
    .selectAll('myOptions')
 	  .data(allGroup)
    .enter()
	  .append('option')
    .text(function (d) {{ return d; }})
    .attr("value", function (d) {{ return d; }})
    dropdownButton.on("change", function(d) {{
    var selected = d3.select(this).property("value");
    (async () => {{
    const channel = await google.colab.kernel.comms.open('my_comm_target', [selected,wordc]);
    let success = false;
    for await (const message of channel.messages) {{
      d3.select("#disp").selectAll("svg").remove()
      let data = message.data;
      console.log(data)
      bar_chart(data) 
    }} ;
  }})()
  }})
 }}
</script>
</body>
</html>
""".format(data = json.dumps(docs))))

comp.windows.x
From: Rick Miller <rick@ee.uwm.edu>
Subject: X-Face?
Organization: Just me.
Lines: 17
Distribution: world
NNTP-Posting-Host: 129.89.2.33
Summary: Go ahead... swamp me.  <EEP!>

I'm not familiar at all with the format of these "X-Face:" thingies, but
after seeing them in some folks' headers, I've *got* to *see* them (and
maybe make one of my own)!

I've got "dpg-view" on my Linux box (which displays "uncompressed X-Faces")
and I've managed to compile [un]compface too... but now that I'm *looking*
for them, I can't seem to find any X-Face:'s in anyones news headers!  :-(

Could you, would you, please send me your "X-Face:" header?

I *know* I'll probably get a little swamped, but I can handle it.

	...I hope.

Rick Miller  <rick@ee.uwm.edu> | <ricxjo@discus.mil.wi.us>   Ricxjo Muelisto
Send a postcard, get one back! | Enposxtigu bildkarton kaj vi ricevos alion!
          RICK MILLER // 16203 WOODS // MUSKEGO, WIS. 53150 // USA

From: kartik@hls.COM (Kartik Chandrasekhar)
S